# Set Paths and logger

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
if AMDOCS_SRC not in sys.path: 
    sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
        
from project.project_generic import Project

import pykhaos.utils.notebooks as nb

import time
start_time = time.time()

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    EXTERNAL_LIB = os.path.join(os.environ.get('BDA_USER_HOME', ''), "lib", "external_libs")
    if EXTERNAL_LIB not in sys.path:
        sys.path.append(EXTERNAL_LIB)
    %load_ext jupyternotify
    
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

20190109-091355 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20190109_091355.log


<IPython.core.display.Javascript object>

# INPUT PARAMS

In [2]:
CCC_TRAIN_TEST_YAML = os.path.join(USECASES_SRC, "churn", "models", "ccc", "input", "ccc_train_test.yaml")

# This name is used when saving the model (Pickle object)
MODEL_NAME = "ccc_comercial_tests" # Results will be stored in a directory named like this

H2O_PORT = 54126 # Be sure no instance running on this port!

# Input data - must be a list but sometime there are problems in the union of h2o dataframes
DP_FILENAME = ["/data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/"]

# Specify the primary key of the model - Usually, msisdn, num_cliente or nif
P_KEY = ["msisdn"] 

# Configuration file for training the h2o model
MODEL_YAML_PATH = os.path.join(USECASES_SRC, "churn", "models", 'ccc', 'input', 'h2o_model.yaml')

# Yaml file of the template for building the excel file
XLSX_REPORT_TEMPLATE_PATH = os.path.join(USECASES_SRC, "churn", "models", "ccc", 'input', 'xlsx_report_template.yaml')

# Save the root path for savings
SAVING_PATH = os.path.join(os.environ.get('BDA_USER_HOME', ''), "data", "churn", "ccc")

# Guarantee directoy is unique
UNIQUE_DIRNAME = dt.datetime.now().strftime("%Y%m%d_%H%M%S")

SAVING_PATH_UNIQUE = os.path.join(SAVING_PATH, "results", MODEL_NAME, UNIQUE_DIRNAME)


# IMPORTS and INITIALIZATION

In [3]:
from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract
sys.path.append(os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel", "pykhaos", "external_lib"))
from pykhaos.modeling.model import Model
import yaml
input_data = yaml.load(open(CCC_TRAIN_TEST_YAML))

Using h2o version 3.18.0.10


# Start H2O

In [4]:
from pykhaos.modeling.h2o.h2o_functions import restart_cluster_loop
restart_cluster_loop(port=H2O_PORT, max_mem_size = "32G") # 

Using h2o version 3.18.0.10
Shutdown 3.18.0.10
init 3.18.0.10
Checking whether there is an H2O instance running at http://localhost:54126..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.7.0_79"; Java(TM) SE Runtime Environment (build 1.7.0_79-b15); Java HotSpot(TM) 64-Bit Server VM (build 24.79-b02, mixed mode)
  Starting server from /opt/cloudera/parcels/Anaconda-2.5.0/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpjfrux_
  JVM stdout: /tmp/tmpjfrux_/h2o_csanc109_started_from_python.out
  JVM stderr: /tmp/tmpjfrux_/h2o_csanc109_started_from_python.err
  Server is running at http://127.0.0.1:54126
Connecting to H2O server at http://127.0.0.1:54126... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Rome
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.10
H2O cluster version age:,7 months and 18 days !!!
H2O cluster name:,H2O_from_python_csanc109_dtrvbb
H2O cluster total nodes:,1
H2O cluster free memory:,28.44 Gb
H2O cluster total cores:,56
H2O cluster allowed cores:,56
H2O cluster status:,"accepting new members, healthy"


Done!
locked=False healthy=True
out of while


# Load Data

In [5]:
%%notify -m "CCC_model_pykahos: ended CCCdata "

from churn.models.ccc.data.ccc_data import  CCC_Data
data_abs_data_obj = CCC_Data(input_data=None, filename=DP_FILENAME, pkey_cols=P_KEY)

!!!!!!! HARD CODED !!!!!!!!!!
set_h2o_training_data - filename ['/data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/']
'/data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/' no existe
Found /data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/. Loading...


20190109-091404 [INFO ] ***** locked=False healthy=True
20190109-091404 [INFO ] Trying to import '/data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/' hdfs=True parquet=False
20190109-091405 [INFO ] Trying to import hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/


Parse progress: |█████████████████████████████████████████████████████████| 100%


20190109-091412 [INFO ] file hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/ found at node 0


file hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_c20181007_c20181107_n60_comercial_msisdn/ found at node 0
**** Preparing __prepare_ccc_data_for_h2o - rows=168085 columns=354
After rgu reduction 89011
About to remove 3 columns
**** Ended __prepare_ccc_data_for_h2o - rows=89011 columns=351


20190109-091413 [INFO ] df_h2o ready to be used!


-------


<IPython.core.display.Javascript object>

# FIT Model

In [56]:
%%notify -m "CCC_model_pykahos: ended fit "
modeler = Model(MODEL_NAME, MODEL_YAML_PATH, verbose=False)
modeler.fit(data_abs_data_obj)

loaded '/var/SP/data/home/csanc109/src/devel/use-cases/churn/models/ccc/input/h2o_model.yaml'
Using test_ratio_split=0.35
(0, 60320)
(1, 28691)
H2O Engine :)
Using h2o version 3.18.0.10
Detected these gridparams:
{'max_after_balance_size': [1.7, 2.5], 'ntrees': [30], 'max_depth': [5]}
**** FIT *****
***** GRIDSEARCH ********
gbm Grid Build progress: |████████████████████████████████████████████████| 100%
    max_after_balance_size max_depth ntrees  \
0                      1.7         5     29   
1                      2.5         5     28   

                                                            model_ids  \
0  Grid_GBM_py_1335_sid_9045_model_python_1547021638053_11556_model_0   
1  Grid_GBM_py_1335_sid_9045_model_python_1547021638053_11556_model_1   

                  auc  
0  0.6344734953647998  
1  0.6343084418201391  

Time in fit method: 00:02:16
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |█████████████████████

,0,1,Error,Rate
0,15177.0,24031.0,0.6129,(24031.0/39208.0)
1,3055.0,15594.0,0.1638,(3055.0/18649.0)
Total,18232.0,39625.0,0.4682,(27086.0/57857.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2977875,0.5351958,235.0
max f2,0.2337654,0.7180703,304.0
max f0point5,0.3536328,0.4702268,152.0
max accuracy,0.3948720,0.6979104,101.0
max precision,0.7241107,1.0,0.0
max recall,0.1108048,1.0,386.0
max specificity,0.7241107,1.0,0.0
max absolute_mcc,0.3093156,0.2290401,218.0
max min_per_class_accuracy,0.3308726,0.6177811,184.0
max mean_per_class_accuracy,0.3223778,0.6222041,197.0


Gains/Lift Table: Avg response rate: 32.23 %



...

metric,threshold,value,idx
max f1,0.2907963,0.5173717,241.0
max f2,0.2046950,0.7095855,331.0
max f0point5,0.3318663,0.4427870,181.0
max accuracy,0.4059002,0.6855941,83.0
max precision,0.7105537,1.0,0.0
max recall,0.0738036,1.0,398.0
max specificity,0.7105537,1.0,0.0
max absolute_mcc,0.3147678,0.1859599,208.0
max min_per_class_accuracy,0.3293472,0.5971959,185.0
max mean_per_class_accuracy,0.3153719,0.5985214,207.0


Gains/Lift Table: Avg response rate: 32.23 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100148,0.4897826,2.2770601,2.2770601,0.7339744,0.7339744,0.0228042,0.0228042,127.7060066,127.7060066
,2,0.0200295,0.4381408,1.8594333,2.0682467,0.5993590,0.6666667,0.0186218,0.0414260,85.9433329,106.8246697
,3,0.0300122,0.4213404,1.5661482,1.9012385,0.5048232,0.6128342,0.0156343,0.0570603,56.6148222,90.1238541
,4,0.0400270,0.4132448,1.6605635,1.8410215,0.5352564,0.5934242,0.0166302,0.0736905,66.0563454,84.1021519
,5,0.0500738,0.4055246,1.6056995,1.7938063,0.5175719,0.5782051,0.0161322,0.0898227,60.5699513,79.3806270
,6,0.1000193,0.3874016,1.4275687,1.6109226,0.4601542,0.5192555,0.0713005,0.1611233,42.7568736,61.0922572
,7,0.1499968,0.3761170,1.3031150,1.5083640,0.4200385,0.4861973,0.0651265,0.2262498,30.3114971,50.8363951
,8,0.2000064,0.3667255,1.2584710,1.4458807,0.4056483,0.4660568,0.0629357,0.2891854,25.8471033,44.5880695
,9,0.2999936,0.3497089,1.2349723,1.3755854,0.3980738,0.4433982,0.1234814,0.4126668,23.4972346,37.5585434
,10,0.4000128,0.3370017,1.1151009,1.3104539,0.3594352,0.4224041,0.1115316,0.5241984,11.5100915,31.0453853




Could not find exact threshold 0.296025101192; using closest threshold found 0.296174504764.
Could not find exact threshold 0.268194021863; using closest threshold found 0.268237061463.
Could not find exact threshold 0.296025101192; using closest threshold found 0.296174504764.
Could not find exact threshold 0.268194021863; using closest threshold found 0.268237061463.
Could not find exact threshold 0.296025101192; using closest threshold found 0.296174504764.
Could not find exact threshold 0.268194021863; using closest threshold found 0.268237061463.
Could not find exact threshold 0.296025101192; using closest threshold found 0.296174504764.
Could not find exact threshold 0.268194021863; using closest threshold found 0.268237061463.
Could not find exact threshold 0.296025101192; using closest threshold found 0.296174504764.
Could not find exact threshold 0.268194021863; using closest threshold found 0.268237061463.
Could not find exact threshold 0.296025101192; using closest thresho

<IPython.core.display.Javascript object>

# SAVE model

In [57]:
%%notify -o

result="FAILED"

import datetime as dt
from churn.utils.constants import YAML_DATA_PREPARATION, YAML_CLOSING_DAY
save_model_path =  os.path.join(SAVING_PATH_UNIQUE, "model")
modeler.save(save_model_path)
result="OK"
"CCC_model_pykahos: [{}] model saved ".format(result)

Using h2o version 3.18.0.10
************** model.py save
['model_fit_config', 'model_fit_params', 'numero_variables', 'target_freq', 'model train metrics', 'target_freq_split', 'test_partition_size', 'model_fit_score_hist', 'model test metrics', 'train_partition_size']
Starting save of model at '/var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model/ccc_comercial_tests.pkl'
save h2o_factory /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model
save h2o_factory_saving_path /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model/h2o_factory
Saved h2o model in path /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model/h2o_factory/Grid_GBM_py_1335_sid_9045_model_python_1547021638053_11556_model_0
Saving object at /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model/ccc_comercial_tests.pkl
Calling to __get

'CCC_model_pykahos: [OK] model saved '

<IPython.core.display.Javascript object>

# Reporter

In [58]:
%%notify -o
result="FAILED"

from pykhaos.reporting.reporter import Reporter

save_reporter_path = os.path.join(SAVING_PATH_UNIQUE, "reporter")

reporter = Reporter(MODEL_NAME, save_model_path, XLSX_REPORT_TEMPLATE_PATH).set_saving_path(save_reporter_path).create()

print 'Informe generado!'

result="OK"
"CCC_model_pykahos: [{}] reporter finished".format(result)

Reporter __init__ model_id=ccc_comercial_tests model_dir=/var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model template_yaml_path=/var/SP/data/home/csanc109/src/devel/use-cases/churn/models/ccc/input/xlsx_report_template.yaml
Loading model /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model/ccc_comercial_tests.pkl
Loading object from /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model/ccc_comercial_tests.pkl
Calling to __setstate H2O Factory
['config', 'params', 'modeler_save_path', 'feature_importance']
<type 'NoneType'>
Calling to __setstate in model
['model_id', 'model_info', 'yaml_filename', 'cols', 'cat_encoding', 'model']
Loaded modeler from path /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/model/h2o_factory/Grid_GBM_py_1335_sid_9045_model_python_1547021638053_11556_model_0
Loading report template from '/var/SP/data/home/cs

/opt/cloudera/parcels/Anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:523: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Writting sheet 'Intro'
Writting sheet 'Split'
Writting sheet 'Model'
Writting sheet 'Train metrics'
Writting sheet 'Test metrics'
Saved in '/var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial_tests/20190109_091356/reporter/ccc_comercial_tests_20190114_121752.xlsx'
Informe generado!


'CCC_model_pykahos: [OK] reporter finished'

<IPython.core.display.Javascript object>

In [59]:
# Uncomment celd if you want to save the sets data (not recommended...)
'''
import h2o
h2o.export_file(modeler.get_sets(), os.path.join(save_model_path, "sets.csv"), force=True)
'''

'\nimport h2o\nh2o.export_file(modeler.get_sets(), os.path.join(save_model_path, "sets.csv"), force=True)\n'

# GENERATE PREDICTIONS

In [19]:
start_time = time.time()
app_name = "CCC_model_predictions_nb"

import pykhaos.utils.pyspark_configuration as pyspark_config
sc, spark, sql_context = pyspark_config.get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = 1, max_n_executors = 10, n_cores = 4,
                         executor_memory = "16g", driver_memory="16g")
print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                     sc.defaultParallelism))

Ended spark session: 118.740973949 secs | default parallelism=2


In [43]:
GENERATE_PREDICTIONS = True

assert(GENERATE_PREDICTIONS)

PREDICT_MODEL_NAME = "ccc_comercial_model"
PREDICT_MODEL_PATH =  os.path.join(SAVING_PATH, "results", "ccc_comercial", "20181229_203318", "model")
closing_day = "20181231" 
print("Program will use the model stored at '{}'".format(PREDICT_MODEL_PATH))

from churn.models.ccc.delivery.delivery_manager import make_predictions
df_lever_predict_hidden = make_predictions(spark, PREDICT_MODEL_NAME, PREDICT_MODEL_PATH, closing_day) # returns a column msisdn (anonymized)

print("Prediction size {}".format(df_lever_predict_hidden.count()))

Program will use the model stored at '/var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial/20181229_203318/model'
Loading model /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial/20181229_203318/model/ccc_comercial_model.pkl
Loading object from /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial/20181229_203318/model/ccc_comercial_model.pkl
Calling to __setstate H2O Factory
['config', 'params', 'modeler_save_path', 'feature_importance']
<type 'NoneType'>
Calling to __setstate in model
['model_id', 'model_info', 'yaml_filename', 'cols', 'cat_encoding', 'model']
Loaded modeler from path /var/SP/data/home/csanc109/data/churn/ccc/results/ccc_comercial/20181229_203318/model/h2o_factory/Grid_GBM_py_303_sid_9108_model_python_1546105225071_16200_model_0
!!!!!!! HARD CODED !!!!!!!!!!
set_h2o_training_data - filename /data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_mobileandfbb
'/data/udf/vf_es/churn/ccc_model/comercial_unl

20190114-085625 [INFO ] ***** locked=False healthy=True
20190114-085625 [INFO ] Trying to import '/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_mobileandfbb' hdfs=True parquet=False
20190114-085626 [INFO ] Trying to import hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_mobileandfbb
20190114-085627 [ERROR] file hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_mobileandfbb could not be found at node 0
20190114-085627 [INFO ] Trying to import hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_mobileandfbb


Parse progress: |█████████████████████████████████████████████████████████| 100%


20190114-085710 [INFO ] file hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_mobileandfbb found at node 1


file hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_mobileandfbb found at node 1
**** Preparing __prepare_ccc_data_for_h2o - rows=3332492 columns=1423
After rgu reduction 3332492
About to remove 208 columns
**** Ended __prepare_ccc_data_for_h2o - rows=3332492 columns=1215


...

Parse progress: |█████████████████████████████████████████████████████████| 100%


20190114-085828 [INFO ] file hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_onlymob found at node 1


file hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial_unlabeled/df_c20181231_n60_comercial_msisdn_onlymob found at node 1
**** Preparing __prepare_ccc_data_for_h2o - rows=1340324 columns=1423
After rgu reduction 1340324
About to remove 208 columns
**** Ended __prepare_ccc_data_for_h2o - rows=1340324 columns=1215


20190114-085848 [INFO ] df_h2o ready to be used!


-------
gbm prediction progress: |████████████████████████████████████████████████| 100%
{'all': {'length': 4672816},
 'mobileandfbb': {'length': 3332492},
 'onlymob': {'length': 1340324}}
['msisdn', 'SCORE', 'reason', 'type']
Prediction size 4672816


In [42]:
from pyspark.sql.types import FloatType
from pykhaos.utils.constants import WHOAMI
from churn.models.ccc.delivery.delivery_manager import PROJECT_NAME

name_table_deanonymized = 'tests_es.{}_tmp_{}_{}_notprepared'.format(WHOAMI, PROJECT_NAME, closing_day)
name_file_delivery = '{}_delivery_{}_{}'.format(PROJECT_NAME, closing_day, dt.datetime.now().strftime("%Y%m%d_%H%M%S"))
# - - - - - - - - - - - - - - - - 
scores_incidences_filename = '/user/csanc109/projects/churn/incidencias/preds_comb_20181231_all_incidences'

from churn.models.ccc.delivery.delivery_manager import merging_process, prepare_delivery, make_predictions, DIR_DOWNLOAD, DIR_DELIVERY
    
merging_process(spark, df_lever_predict_hidden, scores_incidences_filename, name_table_deanonymized)
prepare_delivery(spark, PROJECT_NAME, closing_day, name_table_deanonymized,name_file_delivery,DIR_DOWNLOAD, DIR_DELIVERY)

Rows read from file 4664611
Ended section 2
Created table tests_es.csanc109_tmp_churn_ccc_20181231_notprepared
Filename successfully deleted /var/SP/data/home/csanc109/data/download/churn/ccc/churn_ccc_delivery_20181231_20190111_095637.csv
Downloaded file /var/SP/data/home/csanc109/data/download/churn/ccc/churn_ccc_delivery_20181231_20190111_095637.csv
File /var/SP/data/home/csanc109/data/delivery/ccc/churn_ccc_delivery_20181231_20190111_095637.csv successfully created
To download the file in your local computer, type from your local: 

 scp csanc109@milan-discovery-edge-387:/var/SP/data/home/csanc109/data/delivery/ccc/churn_ccc_delivery_20181231_20190111_095637.csv <local_directoy>
Process ended successfully


# [OPTIONAL] POST PROCESS ANALYSIS - FIXME (pending to implement in h2o to avoid pandas)

In [ ]:
data = data_abs_data_obj.data()
df_data = data.as_data_frame()
data.types

#train = modeler.get_model_info().get("model train metrics")["scores"]
#test = modeler.get_model_info().get("model test metrics")["scores"]

from pykhaos.reporting.utils.plots import univariate_analysis, univariate_hist
df_uni = univariate_analysis(df_data, "TARIFF", target="label", sort=None) # rows=num tarifas || cols [u'TARIFF', u'positivo', u'total', u'frec_loc', u'frec'], dtype='object')

univariate_hist(df_uni, "TARIFF", sort = None)

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
df_ = df_data.copy()

df_pivot_label = pd.pivot_table(df_[["label", "TARIFF"]], columns="label", index="TARIFF", aggfunc=lambda x: len(x))
df_pivot_label.columns=["0", "1"]
import matplotlib.pyplot as plt
ax = df_pivot_label[['0','1']].plot(kind='bar', title ="V comp", figsize=(15, 10), legend=True, fontsize=8)
ax.set_ylabel("V", fontsize=12)
plt.show()

In [ ]:
var1 = "codigo_postal_city"
var2 = "Order_N1_Description"

var = [var1,var2 ]
target = "label"

import numpy as np
from sklearn.preprocessing import MinMaxScaler, Imputer

from reportlab.lib.enums import TA_RIGHT
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image, Spacer, PageBreak, Table

from sklearn.tree import export_graphviz
import pydotplus
from sklearn.externals.six import StringIO
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

df_data = df_data[~df_data[target].isna()]
df_data[var1] = df_data[var1].fillna(-1)
df_data[var2] = df_data[var2].fillna("UNKNOWN")
X = np.array(df_data[var].copy())
le = preprocessing.LabelEncoder()
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)

X[var.index(var1)] = imp.fit_transform(X[var.index(var1)])
#X[var1] = X[var1].isna().astype('int')
le.fit(X[var.index(var2)])
X[var.index(var2)] = le.transform(X[var.index(var2)])

y = np.array(df_data[target])
dtree = DecisionTreeClassifier(max_depth=15, min_samples_leaf=500)
dtree.fit(X, y)
from pykhaos.reporting.utils.plots import prune

#dtree_prune = prune(dtree)
dtree_prune = dtree

from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydotplus


dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,
                filled=True, rounded=True, feature_names=var,
                special_characters=True, rotate=True, class_names=["NO_COMERCIAL", "COMERCIAL"],
                impurity=False, proportion=True, label='root')

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
filename = '/Users/csanc109/Documents/Projects/Churn/ccc_model/results/images/graph_{}.png'.format(int(time.time()))
graph.write_png(filename)

# SHOW IMAGE FROM FILE
from IPython.display import display
from PIL import Image
display(Image.open(filename))

# UNIVARIANT ANALYSIS

In [ ]:
import yaml
from pykhaos.reporting.type.univariate import Univariate
from churn.models.ccc.data.ccc_data import  CCC_Data

pdf_report_template_path = os.path.join(USECASES_SRC, "churn", "models", "ccc", 'input', 'univariante_template.yaml')

print 'univariant analysis: '
#periods = [201805, 201806, 201807]
default_filename = os.path.join(USECASES_SRC, "churn", "models", "ccc", "input", "ccc_train_test.yaml")
input_data = yaml.load(open(default_filename))
data_abs_data_obj = CCC_Data(input_data)
univariate_info = Univariate(data_abs_data_obj, MODEL_NAME+'_univariate').analysis()
univariate_info.save()
print 'end of analysis: {}'.format(MODEL_NAME+'_univariate')

print 'Generando informe: '
reporter = Reporter() \
    .set_model_info(ModelInfo.load(MODEL_NAME+'_univariate')) \
    .set_template('model/univariante.yaml')\
    .set_saving_path(os.path.join(os.environ.get('BDA_USER_HOME', ''), "data", "churn", "ccc", "results"))
reporter.create()
print 'Informe generado!'
